In [1]:
import torch 
import torch.nn as nn
import torch.optim as optim 
import torchvision 
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
])

In [3]:
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True)

Files already downloaded and verified


In [5]:
latent_dim=100
lr=0.0002
beta1=0.5 
beta2=0.999
num_epochs=10 